In [77]:
import sys
import pandas as pd
import scipy
import sklearn.preprocessing
import numpy as np

In [78]:
events = pd.read_parquet("./data/events.parquet")
items = pd.read_parquet("./data/items.parquet")

### Коллаборативная фильтрация: ALS

In [79]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()
train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

In [80]:
# перекодируем идентификаторы пользователей:
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов:
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/3198522641.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/3198522641.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/3198522641.py:13: SettingWithCopyWarn

In [81]:
print(events_train["item_id_enc"].max())

43304


Вычислите размер матрицы user_item_matrix_train, как если бы она хранила все свои элементы, включая пропуски, и для каждого элемента использовался бы один байт. Ответ приведите в виде целого числа гигабайтов, отбросив дробную часть.

In [82]:
# Получаем количество уникальных пользователей и объектов
num_users = events_train["user_id_enc"].nunique()
num_items = events_train["item_id_enc"].nunique()

# Вычисляем размер матрицы в байтах (1 байт на элемент)
matrix_size_bytes = num_users * num_items

# Переводим размер в гигабайты
matrix_size_gb = matrix_size_bytes / (1024 ** 3)

# Отбрасываем дробную часть и выводим целое число
matrix_size_gb_int = int(matrix_size_gb)

In [83]:
print(matrix_size_gb_int)

16


Мы создадим sparse-матрицу в формате CSR. Такая матрица хранит только имеющиеся значения и не резервирует память под пропуски. Это помогает значительно сэкономить оперативную память при работе с разреженными матрицами.

In [84]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [85]:
matrix_size_gb_2 = sum([sys.getsizeof(i) for i in user_item_matrix_train.data]) / 1024 ** 3

In [86]:
print(matrix_size_gb_2)

0.26370687410235405


In [87]:
from scipy import sparse

sparse.save_npz("./data/user_item_matrix_train.npz", user_item_matrix_train)

Имея подготовленную матрицу взаимодействий, перейдём к третьему шагу — создадим ALS-модель. Для примера возьмём количество латентных факторов для матриц $P, Q$, равным 50. 
Выполните код для создания и тренировки модели.

In [88]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

  0%|          | 0/50 [00:00<?, ?it/s]

Чтобы получить рекомендации для пользователя с помощью модели ALS, используем такую функцию:

In [89]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
        user_id_enc,
        user_item_matrix[user_id_enc],
        filter_already_liked_items=not include_seen,
        N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])

    return recommendations

Получим рекомендации для всех имеющихся пользователей. Выполните код ниже.

In [90]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded,
    user_item_matrix_train[user_ids_encoded],
    filter_already_liked_items=False, N=100)

Код возвращает рекомендации как список списков, это не очень удобно. Преобразуем его в более удобный формат — табличный.

In [91]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(),
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

Сохраним полученные рекомендации в файл, они ещё нам пригодятся.

In [92]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("./data/als_recommendations.parquet")

Score от ALS не лежат на той же шкале, что и пользовательские оценки. Сравнивать исходные и новые оценки напрямую — некорректно. Поэтому посчитать метрики MAE, RMSE проблематично. Вместо них можно использовать метрики ранжирования. Они сравнивают не абсолютные значения рейтингов и их оценок, а соответствие порядков. Метрики ранжирования покажут, насколько порядок рекомендаций по убыванию score соответствует порядку объектов по убыванию пользовательских оценок. 


На практике часто используют метрику NDCG, она принимает значение от 0 (предлагаемый порядок никак не соответствует истинному) до 1 (предлагаемый порядок в точности соответствует истинному). 

In [93]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
           .rename(columns={"rating": "rating_test"}),
           on=["user_id", "item_id"], how="left")
)

Подсчитать метрику NDCG для одного пользователя поможет готовая реализация из scikit-learn:

In [94]:
import sklearn.metrics


def compute_ndcg(rating: pd.Series, score: pd.Series, k):
    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """

    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [95]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(
    lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/3704688269.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(


In [96]:
print(ndcg_at_5_scores.mean())

0.9759534535118616


### Контентные рекомендации

преобразуем значения в genre_and_votes из текстового представления в тип в Python:

In [97]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [98]:
def get_genres(items):
    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    genres_counter = {}

    for k, v, in items.iterrows():
        genre_and_votes = v.get('genre_and_votes')
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"

    return genres

In [99]:
genres = get_genres(items)

In [100]:
genres["score"] = genres["votes"] / genres["votes"].sum()

In [101]:
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


In [102]:
# Функция в коде ниже строит матрицу вида «книга-жанр».
def get_item2genre_matrix(genres, items):
    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()

    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []

    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)),
                                         shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)

    return genres_csr

In [103]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Аналогичным образом получим матрицу с весами по жанрам для какого-нибудь пользователя, например, для пользователя с идентификатором 1000010. 

In [104]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)

Сколько получилось существующих элементов в user_items_genres_csr для выбранного пользователя

In [105]:
print(user_items_genres_csr.nnz)

149


In [106]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [107]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.185241
38,Classics,3414934,0.103879
25,Fantasy,6850060,0.072447
5,Nonfiction,1737406,0.050865
24,Science Fiction,1218917,0.040920


Получите наиболее релевантные рекомендации для пользователя. Дополните код так, чтобы переменная top_k_indices заполнялась индексами соответствующих книг. Для этого удобно использовать np.argsort от similarity_scores, подсчитанной для всех книг.

In [108]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[::-1][:k]

In [109]:
print(top_k_indices)

[ 4471 36093 14087  9476  4460]


После вычисления top_k_indices по полученным индексам извлеките список объектов, которые могут быть интересны пользователю

In [110]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
    display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
393210,"G.K. Chesterton, Jonathan Lethem",The Man Who Was Thursday: A Nightmare,"{'Fiction': 1257, 'Classics': 929, 'Mystery': 469, 'Fantasy': 293, 'Philosophy': 156, 'Literatur..."
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"
39408,"Paulo Coelho, Alan R. Clarke, James Noel Smith",The Alchemist,"{'Fiction': 14023, 'Classics': 5787, 'Fantasy': 3289, 'Philosophy': 2759}"


### Валидация

Валидация рекомендательной системы — это процесс оценки, насколько эффективно и точно она предсказывает предпочтения пользователей. Валидация помогает определить, насколько хорошо система выполняет свою задачу.

Посчитаем recall и precision для ALS-рекомендаций (als_recommendations). Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
- gt (ground truth): объект есть в тестовой выборке;
- pr (predicted): объект есть в рекомендациях.

Теперь разметим признаки бинарной классификации:
- TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
- FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
- FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

Ниже приведён код для такой разметки:

In [111]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):
    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")

    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[
        events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)

    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]],
        on=["user_id", "item_id"], how="outer")

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()

    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [112]:
# Обработаем ALS-рекомендации для подсчёта метрик для 5 лучших рекомендаций:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
    events_train,
    events_test,
    als_recommendations,
    top_k=5
)

Common users: 123223


/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/2783672476.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True
/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/2783672476.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events_recs_common["gt"] = events_recs_common["gt"].fillna(False)


In [113]:
# Функции compute_cls_metrics для расчёта recall.
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [114]:
# Пример использования функции
precision_at_5, recall_at_5 = compute_cls_metrics(events_recs_for_binary_metrics)
print(f"Precision@5: {precision_at_5:.3f}, Recall@5: {recall_at_5:.3f}")

Precision@5: 0.008, Recall@5: 0.014


In [115]:
events_recs_for_binary_metrics_10 = process_events_recs_for_binary_metrics(
    events_train,
    events_test,
    als_recommendations,
    top_k=10
)

/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/2783672476.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_76659/2783672476.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events_recs_common["gt"] = events_recs_common["gt"].fillna(False)


In [116]:
precision_at_10, recall_at_10 = compute_cls_metrics(events_recs_for_binary_metrics_10)
print(f"Precision@10: {precision_at_10:.3f}, Recall@10: {recall_at_10:.3f}")

Precision@10: 0.009, Recall@10: 0.031


In [126]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items + 1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]
similar_items = pd.DataFrame(columns=["item_id_enc", "sim_item_id_enc", "score"])

# Convert columns to the appropriate data types
similar_items["score"] = similar_items["score"].astype(float)
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype(int)

similar_items_list = []
for item_id, sim_item_ids, sim_scores in zip(train_item_ids_enc, sim_item_item_ids_enc.tolist(),
                                             sim_item_scores.tolist()):
    similar_items_list.append((item_id, sim_item_ids, sim_scores))

# Create DataFrame from the list
similar_items = pd.DataFrame(similar_items_list, columns=["item_id_enc", "sim_item_id_enc", "score"])

similar_items = similar_items.explode(["sim_item_id_enc", "score"], ignore_index=True)

# Convert columns to the appropriate data types
similar_items["score"] = similar_items["score"].astype(float)
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype(int)

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])

In [127]:
# получаем изначальные идентификаторы
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])

In [128]:
# убираем ненужные колонки
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

KeyError: "['item_id_enc'] not found in axis"

In [129]:
# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [130]:
# Выводим результат
print(similar_items.head())
print(similar_items[similar_items["item_id_1"] == 7126].index)

      score  item_id_1  item_id_2
1  0.922487      22034      22026
2  0.874762      22034       6882
3  0.873764      22034      22028
4  0.850658      22034     364089
5  0.835735      22034       9827
Index([25873, 25874, 25875, 25876, 25877, 25878, 25879, 25880, 25881, 25882], dtype='int64')


In [131]:
similar_items.to_parquet("./data/similar_items.parquet")

In [132]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si) 


,item_id,author,title,genre_and_votes,average_rating,ratings_count
1909078,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,"{'Classics': 15812, 'Fiction': 8028, 'Cultural...",4.19,390293


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
66760,0.964478,7144,12505,"Fyodor Dostoyevsky, Anna Brailovsky, Constance...",The Idiot,"{'Classics': 4036, 'Fiction': 2576}",4.18,76392
66761,0.953918,7144,12857,"Fyodor Dostoyevsky, Constance Garnett",The Gambler,"{'Classics': 946, 'Fiction': 729, 'Cultural-Ru...",3.88,22024
66762,0.952008,7144,67326,Fyodor Dostoyevsky,Poor Folk,"{'Classics': 320, 'Fiction': 235, 'Literature-...",3.73,4957
66763,0.946848,7144,5508624,Leo Tolstoy,Family Happiness,"{'Classics': 140, 'Fiction': 112, 'Cultural-Ru...",3.85,3337
66764,0.939763,7144,4934,"Fyodor Dostoyevsky, Fyodor Dostoyevsky, Richar...",The Brothers Karamazov,"{'Classics': 7496, 'Fiction': 5491, 'Cultural-...",4.31,158410
66765,0.938016,7144,17877,"Fyodor Dostoyevsky, Constance Garnett",The House of the Dead,"{'Classics': 533, 'Fiction': 441, 'Cultural-Ru...",4.04,8548
66766,0.937005,7144,929782,"Jack London, Andrew Sinclair",Martin Eden,"{'Classics': 435, 'Fiction': 405, 'Literature-...",4.39,13257
66767,0.936360,7144,28382,Nikolai Gogol,Diary of a Madman and Other Stories,"{'Classics': 284, 'Fiction': 243, 'Short Stori...",4.09,6241
66768,0.936321,7144,17690,"Franz Kafka, Max Brod, Willa Muir, Edwin Muir",The Trial,"{'Classics': 4607, 'Fiction': 4173, 'Literatur...",3.98,135862
66769,0.934541,7144,63038,Victor Hugo,The Man Who Laughs,"{'Classics': 352, 'Fiction': 176, 'Cultural-Fr...",4.22,5449


In [132]:
print_sim_items(7144, similar_items)